# Pandas Challenge - PyCity Schools

Martin Singla - Data Analyst 

Data Analytics Bootcamp - Tecnológico de Monterrey / Trinity Education

In the following Jupyter notebook we utilize Python's Pandas dependendency to analyze education trends at a school district level. 

### Loading & data wrangling 

In [134]:
#Load dependencies and data
import pandas as pd

schools = pd.read_csv("Resources/schools_complete.csv", encoding = "utf-8")
students = pd.read_csv("Resources/students_complete.csv", encoding = "utf-8")

In [215]:
#Checking data frames: Schools
schools.count() #check for NaN
schools.head() #14 schools with its name, type, size and budget

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [136]:
#Checking data frames: Students
students.count()
students.head() #39,170 students w/ its name, gender, grade, school, reading and math scores.

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [137]:
#Merging datasets
data = pd.merge(students, schools, on= "school_name", how= "left")
data = data.rename(columns= {"size" : "Tot.Students.in.School",
                             "budget" : "Tot.School.Budget"})
data.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,Tot.Students.in.School,Tot.School.Budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


### District Level Summary Statistics

In [139]:
#District Education statistics summary 

#Creating df
df1 = pd.DataFrame({
    "Total Schools": [schools["school_name"].count()] ,
    "Total Students": [students["student_name"].count()], 
    "Total Budget": [schools["budget"].sum()], 
    "Avg.Math Score": [students["math_score"].mean()], 
    "Avg.Reading Score": [students["reading_score"].mean()], 
    "% Passing Math": [students.loc[students["math_score"] >= 70,"math_score"].count() / students["math_score"].count() * 100], 
    "% Passing Reading": [students.loc[students["reading_score"] >= 70,"reading_score"].count() / students["math_score"].count() * 100], 
    "% Overall Passing": [students.loc[(students["reading_score"] >= 70) & (students["math_score"] >= 70), "Student ID"].count() / students["Student ID"].count()*100]
})

#Formatting df
df1_formated = df1.copy()
df1_formated["Total Students"] = df1_formated["Total Students"].map("{:,}".format)
df1_formated["Total Budget"] = df1_formated["Total Budget"].map("${:,}".format)
df1_formated["Avg.Math Score"] = df1_formated["Avg.Math Score"].map("{:.2f}".format)
df1_formated["Avg.Reading Score"] = df1_formated["Avg.Reading Score"].map("{:.2f}".format)
df1_formated["% Passing Math"] = df1_formated["% Passing Math"].map("%{:.2f}".format)
df1_formated["% Passing Reading"] = df1_formated["% Passing Reading"].map("%{:.2f}".format)
df1_formated["% Overall Passing"] = df1_formated["% Overall Passing"].map("%{:.2f}".format)

df1_formated

,Total Schools,Total Students,Total Budget,Avg.Math Score,Avg.Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428",78.99,81.88,%74.98,%85.81,%65.17


Top level summary statistics indicate that students are perfroming better at reading classes than math classes. From my own experience back in high school, I would consider this trend accurate!

### School Level Summary Statistics

In [159]:
#Summary statistics at a school level
data_group = data.groupby("school_name")
#Creating df
df2 = pd.DataFrame({
    "School Type": data_group["type"].first(),
    "Total Students": data_group["Tot.Students.in.School"].first(),
    "Total School Budget": data_group["Tot.School.Budget"].first(),
    "Per Student Budget": data_group["Tot.School.Budget"].first() / data_group["Tot.Students.in.School"].first(),
    "Average Math Score": round(data_group["math_score"].mean(), 2),
    "Average Reading Score": round(data_group["reading_score"].mean(), 2),
    "% Passing Math": data.loc[data["math_score"] >= 70]
                        .groupby("school_name")["math_score"]
                        .count() / data_group["Tot.Students.in.School"].first() * 100,
    "% Passing Reading": data.loc[data["reading_score"] >= 70].groupby("school_name")["reading_score"].count() / data_group["Tot.Students.in.School"].first() * 100,
    "% Overall Passing": data.loc[(data["math_score"] >= 70) & (data["reading_score"] >= 70)]
                        .groupby("school_name")["math_score"]
                        .count() / data_group["Tot.Students.in.School"].first() * 100
})
#Formatting df
df2_format = df2.copy()
df2_format["Total Students"] = df2_format["Total Students"].map("{:,}".format)
df2_format["Total School Budget"] = df2_format["Total School Budget"].map("${:,.2f}".format)
df2_format["Per Student Budget"] = df2_format["Per Student Budget"].map("${:,.2f}".format)
df2_format["% Passing Math"] = df2_format["% Passing Math"].map("%{:.2f}".format)
df2_format["% Passing Reading"] = df2_format["% Passing Reading"].map("%{:.2f}".format)
df2_format["% Overall Passing"] = df2_format["% Overall Passing"].map("%{:.2f}".format)

df2_format

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.05,81.03,%66.68,%81.93,%54.64
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,%94.13,%97.04,%91.33
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,%65.99,%80.74,%53.20
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.10,80.75,%68.31,%79.30,%54.29
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,%93.39,%97.14,%90.60
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29,80.93,%66.75,%80.86,%53.53
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,%92.51,%96.25,%89.23
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,%65.68,%81.32,%53.51
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,%66.06,%81.22,%53.54


There are some clear differences between schools with a very high % of overall passing students (above 90%) and some schools with very bad perfromance (below 55%). Actually, there are not many schools with overall % of passing students between 55% and 90%. School performance in the district is very polarized!! Maybe some correlation statistics and regression analysis using budget per student data will help further understand this trend.

### Top & Worst Performing Schools

In [160]:
#Top 5 Performing Schools in terms of "% Overall passing students"
df2_format.sort_values("% Overall Passing", ascending= False).head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,%94.13,%97.04,%91.33
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.42,83.85,%93.27,%97.31,%90.95
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,%93.39,%97.14,%90.60
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.27,83.99,%93.87,%96.54,%90.58
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,%94.59,%95.95,%90.54


In [161]:
#Worst 5 Performing Schools in terms of "% Overall passing students"
df2_format.sort_values("% Overall Passing", ascending= True).head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.84,80.74,%66.37,%80.22,%52.99
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,%65.99,%80.74,%53.20
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,%65.68,%81.32,%53.51
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29,80.93,%66.75,%80.86,%53.53
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,%66.06,%81.22,%53.54


### Average Math & Reading Scores By Grade & School

In [162]:
pd.unique(data["grade"]) #check grades listed = 9th, 10th, 11th and 12th

#Average Math Scores by grade and school
df3 = pd.DataFrame({
    "9th Grade": round(data.loc[data["grade"] == "9th"].groupby("school_name")["math_score"].mean(), 2),
    "10th Grade": round(data.loc[data["grade"] == "10th"].groupby("school_name")["math_score"].mean(), 2),
    "11th Grade": round(data.loc[data["grade"] == "11th"].groupby("school_name")["math_score"].mean(), 2),
    "12th Grade": round(data.loc[data["grade"] == "12th"].groupby("school_name")["math_score"].mean(), 2)
})
df3

,9th Grade,10th Grade,11th Grade,12th Grade
school_name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


In [146]:
#Average Reading Scores by grade and school
df3 = pd.DataFrame({
    "9th Grade": round(data.loc[data["grade"] == "9th"].groupby("school_name")["reading_score"].mean(), 2),
    "10th Grade": round(data.loc[data["grade"] == "10th"].groupby("school_name")["reading_score"].mean(), 2),
    "11th Grade": round(data.loc[data["grade"] == "11th"].groupby("school_name")["reading_score"].mean(), 2),
    "12th Grade": round(data.loc[data["grade"] == "12th"].groupby("school_name")["reading_score"].mean(), 2)
})
df3

,9th Grade,10th Grade,11th Grade,12th Grade
school_name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


### Student Performance by School Spending

In [177]:
#Create groups/bins for budget per student
(schools["budget"]/schools["size"]).describe() #check distribution
((schools["budget"]/schools["size"]).max() - (schools["budget"]/schools["size"]).min())/ 4 #bins of 19.25 -> aprox ~20

bins = [0, 595, 615, 635, 1000]
labs = ["<$595 per/student",
       "$595-615 per/student",
       "$615-635 per/student",
       "$635< per/student"]

df2["Per.Student.Budget.Group"] =pd.cut(data_group["Tot.School.Budget"].first() / data_group["Tot.Students.in.School"].first(), 
                                        bins, labels= labs)
df4 = pd.DataFrame({
    "Avg.Math Score": round(df2.groupby("Per.Student.Budget.Group")["Average Math Score"].mean(), 2),
    "Avg.Reading Score": round(df2.groupby("Per.Student.Budget.Group")["Average Reading Score"].mean(), 2),
    "% Passing Math": round(df2.groupby("Per.Student.Budget.Group")["% Passing Math"].mean(), 2),
    "% Passing Reading": round(df2.groupby("Per.Student.Budget.Group")["% Passing Reading"].mean(), 2),
    "% Overall Passing": round(df2.groupby("Per.Student.Budget.Group")["% Overall Passing"].mean(), 2)
})
df4

,Avg.Math Score,Avg.Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Per.Student.Budget.Group,,,,,
<$595 per/student,83.45,83.94,93.46,96.61,90.37
$595-615 per/student,83.60,83.88,94.23,95.90,90.22
$615-635 per/student,80.20,82.42,80.04,89.54,72.62
$635< per/student,77.87,81.37,70.35,83.00,58.86


Interestingly enough, there seems to be a negative correlation between budget per student groups and average student performance per shcool. Intuitivelly one would expect that at higher budget, higher perfromance (at reading, math, overall passing, etc.). However, we found the inverse correlation.

However, this may be due to the amount of students per school. School budget allocation may not be directly related to the amount of students studying in that school, but to other variables, so naturally, larger studentship in a particular school will average at a lower per-student average budget. What this table seems to show is that students in schools with larger amount of students inscripted seem to be perfroming better than students in small schools (although that may translate in higher budget per student).

Another possible trend may be that the due to low student perfromance in certain schools, the school district authority started allocating a higher budget to those schools to try to revert this trend. That immediately translated in a higher budget-per-student , but not immediately in school performance, as those processess take time, maybe years.

Let´s explore those other variables to see what´s happening...


### Student Performance by School Type

In [178]:
#Student Perfromance by school type

df5 = pd.DataFrame({
    "Avg.Math Score": round(df2.groupby("School Type")["Average Math Score"].mean(), 2),
    "Avg.Reading Score": round(df2.groupby("School Type")["Average Reading Score"].mean(), 2),
    "% Passing Math": round(df2.groupby("School Type")["% Passing Math"].mean(), 2),
    "% Passing Reading": round(df2.groupby("School Type")["% Passing Reading"].mean(), 2),
    "% Overall Passing": round(df2.groupby("School Type")["% Overall Passing"].mean(), 2)
})
df5

,Avg.Math Score,Avg.Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.47,83.90,93.62,96.59,90.43
District,76.96,80.97,66.55,80.80,53.67


Charter schools have a clear better performance at all metrics than District schools.

In [212]:
#Just to cross validate, let´s check budget allocation per student in relation to school type
pd.DataFrame({
    "Charter": [round(df2.loc[df2["School Type"] == "Charter", "Per Student Budget"].mean(),2),
               round(df2.loc[df2["School Type"] == "Charter", "Total School Budget"].mean(),2)],
    "District": [round(df2.loc[df2["School Type"] == "District", "Per Student Budget"].mean(),2),
                round(df2.loc[df2["School Type"] == "District", "Total School Budget"].mean(),2)]
}, index= ["Avg. Budget per Student", "Avg. Tot Budget per School"])

,Charter,District
Avg. Budget per Student,599.50,643.57
Avg. Tot Budget per School,912688.12,2478274.71


Average budget per student allocation in district schools as well as total average budget per school seems to be higher than charter schools, but charter schools still have the highest grades. 

In [208]:
#Amount of students per school type
pd.DataFrame({
    "Charter": [data.loc[data["type"] == "Charter", "Student ID"].count()],
    "District": [data.loc[data["type"] == "District", "Student ID"].count()]
})

,Charter,District
0,12194,26976


Still the majority of students go to district schools... and that District schools have the worst perfroamnce independently of the larget budget allocation..

### School Performance by School Size

In [213]:
#Student Perfromance by school size
df2["Total Students"].describe() # Minimum is 427 students, max 4976 students, median 2283
bins = [0,1500, 3000, 5000]
labs = ["Small (<1500 students)",
       "Medium (1500<3000 students)",
       "Large (3000< students)"]
df2["School.size.group"] = pd.cut(df2["Total Students"], bins, labels= labs)
df6 = pd.DataFrame({
    "Avg.Math Score": round(df2.groupby("School.size.group")["Average Math Score"].mean(), 2),
    "Avg.Reading Score": round(df2.groupby("School.size.group")["Average Reading Score"].mean(), 2),
    "% Passing Math": round(df2.groupby("School.size.group")["% Passing Math"].mean(), 2),
    "% Passing Reading": round(df2.groupby("School.size.group")["% Passing Reading"].mean(), 2),
    "% Overall Passing": round(df2.groupby("School.size.group")["% Overall Passing"].mean(), 2)
})
df6

,Avg.Math Score,Avg.Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School.size.group,,,,,
Small (<1500 students),83.66,83.89,93.50,96.45,90.12
Medium (1500<3000 students),80.90,82.82,83.56,90.59,76.76
Large (3000< students),77.06,80.92,66.46,81.06,53.67


Verified... Smaller size schools have systematically better student performance than large schools.  Small size studentship schools are charter schools in general. Those schools have independence to define their curriculum as well as giving more hours of classess and defining a maximum amount of students enrolled in their programs. Consequently, those schools have better perfromance per student. One may think... those charter schools may be located in better well-off neighbourhoods, and their studentship may come from upper well-educated socio-economic groups. Such conditions immediately translate into higher perfromance per student. 
Contrary, district schools are less but have massive studentship. Large studentship may cause that teachers are less individually focused and dedicated into the success of their particular students. Students from district schools may also come from tougher neighbourhoods , or there might be other reasons why large studenship schools (district schools in most cases) have systematically worst perfromance.

Budget allocation is not directly related, but contrary to that, small size schools (notably charter schools) have less budget allocation on average (both in absolute terms and per-student). This means that there is not a direct correlation between student perfromance and budget allocation in the short term, and that budget allocation is not proportional to students per school. The higher budget in low performing schools may be a result of a particular education policy developed by the school district office  with the objective or reverting the trend, but the results will only be visible in some years time. 

